In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
## 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
## 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
## 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
## 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
## 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [4]:
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

## 指数的计算
# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(img):
    image = ee.Image(img).clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

# RF训练

In [5]:
### 将人工选取样本的影像变成影像集
## 导入人工采集的样本
manual_samples = ee.FeatureCollection('users/311605001111/TIBET_2015')
print('manual-sample number:{}'.format(manual_samples.size().getInfo()))
# 水体的样本点数
samplepoint_water = manual_samples.filter(ee.Filter.eq('label',1))
print('water sample number:{}'.format(samplepoint_water.size().getInfo()))
# 非水体的样本点数
samplepoint_land = manual_samples.filter(ee.Filter.eq('label',0))
print('nowater sample number:{}'.format(samplepoint_land.size().getInfo()))


image_id = ee.List(manual_samples.distinct('Image_id').aggregate_array('Image_id'))
image_set = ee.List([])
for i in image_id.getInfo():
    if 'LT05' in i:
        image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
    elif 'LE07' in i:
        image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
    else:
        image_set = image_set.add(ee.Image(str(i)).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
# 图像集，image_col
image_col = ee.ImageCollection(image_set)
Map.addLayer(image_col.first().clip(roi), visParams,'image')
print(image_col.aggregate_array('system:id').getInfo())

manual-sample number:2090
water sample number:505
nowater sample number:1585
['LANDSAT/LC08/C01/T1_SR/LC08_139036_20150907', 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150822', 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150705', 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150416', 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20151102', 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20151001']


In [6]:
## 导入自动采集的样本
samples = ee.FeatureCollection('users/311605001111/tibetan_2015')
print('auto-sample number:{}'.format(samples.size().getInfo()))

label = 'waterclass'
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

auto_samples = k_mean(samples)
print("筛选后，剩余的样本数： ",auto_samples.size().getInfo())

trainedClassifier = ee.Classifier.smileRandomForest(20).train(auto_samples,label,bands)

auto-sample number:5578
筛选后，剩余的样本数：  5565


# 精度验证函数

In [13]:
def image_accuracy(img):
    image = maskSR(water_index(img)).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
    manual_point = manual_samples.filter(ee.Filter.eq('Image_id',img.get('system:id')))
    water_image = ee.Image.constant(1).toFloat().rename('label').clipToCollection(manual_point.filter(ee.Filter.eq('label',1)))
    land_image = ee.Image.constant(0).toFloat().rename('label').clipToCollection(manual_point.filter(ee.Filter.eq('label',0))) 
    water_samples = water_image.sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 4800,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    land_samples = land_image.sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 7800,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    all_samples = ee.FeatureCollection([water_samples,land_samples]).flatten()
    inter_points = image.sampleRegions(**{
        'collection': all_samples,
        'properties': ['label'],
        'scale': 30,
        'geometries': True,
    })
    inter_samples = inter_points.map(lambda i : i.setMulti({'Image_id':img.get('system:id')}))
    return inter_samples

In [14]:
samples_dataset = image_col.map(image_accuracy).flatten()
print(samples_dataset.size().getInfo())
print(samples_dataset.first().getInfo())
# print(samples_dataset.distinct('Image_id').aggregate_array('Image_id').getInfo())
# print(samples_dataset.distinct('Image_id').size().getInfo())

807
{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [90.50782751534706, 34.88775711465849]}, 'id': 'LC08_139036_20150907_0_38_0', 'properties': {'Image_id': 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150907', 'label': 1, 'waterclass': 1}}


In [15]:
# 利用误差矩阵进行验证
test_accuracy = samples_dataset.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

[[402, 0], [0, 405]]
total accuracy:1
kappa:1


In [ ]:
miss = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
print(miss.size().getInfo())
print(miss.first().getInfo())
print(miss.distinct('Image_id').aggregate_array('Image_id').getInfo())

# 附录

In [ ]:
id = 'LANDSAT/LC08/C01/T1_SR/LC08_123039_20200209'
img = maskL8sr(water_index(ee.Image(id)))
Map.addLayer(img.clip(roi), visParams,'image')

In [ ]:
munal_sample = ee.FeatureCollection('users/311605001111/WUHAN_2020').filter(ee.Filter.eq('Image_id','LANDSAT/LC08/C01/T1_SR/LC08_122039_20200218'))
empty = ee.Image().byte()
outlines = empty.paint(munal_sample,'label',1);
Map.addLayer(outlines, {'palette': ['red', 'blue'],'min':0, 'max':1}, 'different color edges')

In [ ]:
# RF分类图像
image = maskL8sr(img).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
# 采样图像
man_point = ee.FeatureCollection('users/311605001111/WUHAN_2020').filter(ee.Filter.eq('Image_id',id))
water_image = ee.Image.constant(1).toFloat().rename('label')
land_image = ee.Image.constant(0).toFloat().rename('label') 
water = water_image.clipToCollection(man_point.filter(ee.Filter.eq('label',1)))
land = land_image.clipToCollection(man_point.filter(ee.Filter.eq('label',0)))
water_samples = water.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 6000,
    'seed': 0,
    'geometries': True,
})
Map.addLayer(water_samples,{'color': 'yellow', 'pointSize': 100,'fillColor': 'yellow'},'water_samples')
print(water_samples.size().getInfo())
land_samples = land.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 100000,
    'seed': 0,
    'geometries': True,
})
Map.addLayer(land_samples,{'color': 'blue', 'pointSize': 100,'fillColor': 'blue'},'land_samples')
print(land_samples.size().getInfo())

In [ ]:
inter_points = image.sampleRegions(**{
    'collection': validation_points,
    'properties': ['label'],
    'scale': 30,
    'geometries': True,
})
print(inter_points.size().getInfo())
# 利用误差矩阵进行验证
test_accuracy = inter_points.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))